In [1]:
from datetime import datetime,date
import logging
import threading
from queue import Queue
from kiteconnect import KiteTicker, KiteConnect
import time
from collections import defaultdict, deque
import csv
import os
import pandas as pd
from Bardata import BarData
import dotenv
from dotenv import load_dotenv,set_key
import argparse
from black_scholes_model import option_greeks
load_dotenv()

True

In [2]:
class DeltaStrategy():

    def __init__(self, config):
        self.config = config
        self.api_key = os.getenv('api_key')
        self.access_token = os.getenv('access_token')
        self.kws = KiteTicker(self.api_key, self.access_token)
        self.kite = KiteConnect(api_key=self.api_key)
        self.instruments = pd.DataFrame(self.kite.instruments(self.config['exchange']))

        self.index_tokens = {'NIFTY':256265, 'SENSEX': 265}
        self.instrument_tokens = [self.index_tokens[self.config['trading_symbol']]]

        self.latest_ticks = {self.index_tokens[self.config['trading_symbol']]: None}
        self.strikes = {}
        self.trading_symbols = {}
        self.expiry = None


        self.tick_queue = Queue()
        self.lock = threading.Lock()

        self.iv = {}
        self.delta = {}
        self.gamma = {}


        return

    def process_ticks(self):
        while True:
            try:
                tick = self.tick_queue.get(timeout=1)
                with self.lock:
                    # Process the tick data
                    print(f"Processing tick: {tick['last_price']}")
                    if tick['instrument_token'] in self.index_tokens.values():
                        self.latest_ticks[tick['instrument_token']] = tick
                        continue
                    
                    tick['trading_symbol'] = self.trading_symbols[tick['instrument_token']]
                    time_to_expiry = (self.expiry - tick['exchange_timestamp']).total_seconds() / (365*86400)  # Convert to days

                    iv, delta, gamma = option_greeks(
                        spot=self.latest_ticks[self.index_tokens[self.config['trading_symbol']]]['last_price'],
                        strike=self.strikes[tick['instrument_token']],
                        dte=time_to_expiry,
                        rate=0.06,  # Example risk-free rate
                        option_type='CE' if 'CE' in tick['tradingsymbol'] else 'PE',
                        option_price=tick['last_price']
                    )

                    self.iv[self.trading_symbols[tick['instrument_token']]] = iv
                    self.delta[tick['trading_symbol']] = delta
                    self.gamma[tick['trading_symbol']] = gamma

                    tick['iv'] = iv
                    tick['delta'] = delta
                    tick['gamma'] = gamma

                    self.latest_ticks[tick['instrument_token']] = tick

                    
                    pass

            except Exception as e:
                print(f"Error processing tick: {e}")    
    
    def update_trading_symbol(self):
        expiry = sorted(self.instruments.loc[self.instruments['name']==self.config['trading_symbol'],'expiry'])
        weekly = expiry[0]
        self.expiry = weekly

        self.lot_size = self.instruments.loc[(self.instruments['name']==self.config['trading_symbol']),'lot_size'].values[0]
        instrument_tokens = self.instruments.loc[(self.instruments['name']==self.config['trading_symbol'])&(self.instruments['expiry']==weekly)&((self.instruments['strike']>self.latest_ticks[self.index_tokens[self.config['trading_symbol']]]['last_price']-(5*self.config['strike_gap']))&(self.instruments['strike']<=self.latest_ticks[self.index_tokens[self.config['trading_symbol']]]['last_price']+(5*self.config['strike_gap'])))&(self.instruments['instrument_type']!='FUT'),'instrument_token'].tolist()
        self.instrument_tokens.extend(instrument_tokens)

        for token in self.instrument_tokens:
            self.trading_symbols[token] = self.instruments.loc[self.instruments['instrument_token']==token,'tradingsymbol'].values[0]
            self.strikes[token] = float(self.instruments.loc[self.instruments['instrument_token']==token,'strike'].values[0])

    def on_ticks(self, ws, ticks):
        
        if self.latest_ticks[self.index_tokens[self.config['trading_symbol']]] is not None:
            self.update_trading_symbol()
            ws.subscribe(self.instrument_tokens)

        for tick in ticks:
            self.tick_queue.put(tick)
        return
    
    def on_connect(self,ws,response):
        """Callback when WebSocket connects."""
        print("WebSocket connected!")
        ws.subscribe(self.instrument_tokens)
        ws.set_mode(ws.MODE_FULL, self.instrument_tokens)
        return
    

    def start(self):
        """Start the WebSocket connection."""
        self.kws.on_ticks = self.on_ticks
        self.kws.on_connect = self.on_connect

        # Start the tick processing thread
        tick_thread = threading.Thread(target=self.process_ticks)
        tick_thread.daemon = True
        tick_thread.start()

        # Start the WebSocket connection
        print("Connecting websocket")
        self.kws.connect(threaded=True)

        try:
            while True:
                time.sleep(5)  # Keep the main thread alive
                print(self.iv)
                print(self.delta)
                print(self.gamma)
        except KeyboardInterrupt:
            logging.info("Stopping the reactor...")
            self.running = False

        return
    


In [3]:
config = {    
    'exchange': 'NFO',
    'trading_symbol': 'NIFTY',
    'strike_gap': 50,
}

In [ ]:
sample = DeltaStrategy(config)
sample.start() 
# sample.instrument_tokens
# print(sample.api_key,sample.access_token)

Connecting websocket
WebSocket connected!
Processing tick: 25475.4


Unhandled Error
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/twisted/internet/selectreactor.py", line 165, in _doReadOrWrite
    why = getattr(selectable, method)()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/twisted/internet/tcp.py", line 250, in doRead
    return self._dataReceived(data)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/twisted/internet/tcp.py", line 255, in _dataReceived
    rval = self.protocol.dataReceived(data)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/twisted/protocols/tls.py", line 339, in dataReceived
    self._flushReceiveBIO()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/twisted/protocols/tls.py", line 305, in _flushReceiveBIO
    ProtocolWrapper.dataReceived(self, bytes)
  File "/Library/Frameworks/Python.framewo

Error processing tick: 
Error processing tick: 
Error processing tick: 
Error processing tick: 
{}
{}
{}
Error processing tick: 
Error processing tick: 
Error processing tick: 
Error processing tick: 
Error processing tick: 
{}
{}
{}
Error processing tick: 
Error processing tick: 
Error processing tick: 
Error processing tick: 
Error processing tick: 
{}
{}
{}
Error processing tick: 
Error processing tick: 
Error processing tick: 
Error processing tick: 


Error processing tick: 
Error processing tick: 


In [1]:
delt={'NIFTY2571025350PE': 0.2735687455155519, 'NIFTY2571025300PE': 0.21022839150286687, 'NIFTY2571025250PE': 0.15979835098549344, 'NIFTY2571025300CE': 0.7677731615908117, 'NIFTY2571025650PE': 0.7945728995606421, 'NIFTY2571025600PE': 0.718378214482524, 'NIFTY2571025650CE': 0.23068854911682907, 'NIFTY2571025450CE': 0.5553307071376501, 'NIFTY2571025400CE': 0.6374992569587192, 'NIFTY2571025450PE': 0.43986259422327567, 'NIFTY2571025600CE': 0.3006729073704144, 'NIFTY2571025350CE': 0.7073716179578223, 'NIFTY2571025550PE': 0.6282175468880942, 'NIFTY2571025500CE': 0.46867268372902515, 'NIFTY2571025400PE': 0.35079587743213014, 'NIFTY2571025700PE': 0.861915056630129, 'NIFTY2571025500PE': 0.5337294305064619, 'NIFTY2571025250CE': 0.8122607946852025, 'NIFTY2571025550CE': 0.38226189018688805, 'NIFTY2571025700CE': 0.17442734592812942}
# Sort the dictionary by values, putting 'nan' values at the end
sorted_items = sorted(delt.items(), key=lambda x: float('-inf') if x[1] == 'nan' else float(x[1]), reverse=True)
sorted_delt = {k: v for k, v in sorted_items}
sorted_delt



{'NIFTY2571025700PE': 0.861915056630129,
 'NIFTY2571025250CE': 0.8122607946852025,
 'NIFTY2571025650PE': 0.7945728995606421,
 'NIFTY2571025300CE': 0.7677731615908117,
 'NIFTY2571025600PE': 0.718378214482524,
 'NIFTY2571025350CE': 0.7073716179578223,
 'NIFTY2571025400CE': 0.6374992569587192,
 'NIFTY2571025550PE': 0.6282175468880942,
 'NIFTY2571025450CE': 0.5553307071376501,
 'NIFTY2571025500PE': 0.5337294305064619,
 'NIFTY2571025500CE': 0.46867268372902515,
 'NIFTY2571025450PE': 0.43986259422327567,
 'NIFTY2571025550CE': 0.38226189018688805,
 'NIFTY2571025400PE': 0.35079587743213014,
 'NIFTY2571025600CE': 0.3006729073704144,
 'NIFTY2571025350PE': 0.2735687455155519,
 'NIFTY2571025650CE': 0.23068854911682907,
 'NIFTY2571025300PE': 0.21022839150286687,
 'NIFTY2571025700CE': 0.17442734592812942,
 'NIFTY2571025250PE': 0.15979835098549344}

In [ ]:
# {25450.0: 0.4976724223363284, 
#  25350.0: 0.3623778248398597, 
#  25200.0: 0.2007679453097071, 
#  25300.0: 0.6875216672977302, 
#  25250.0: 0.7382844552795542, 
#  25650.0: 0.24475418571206192, 
#  25400.0: 0.5654359970186535, 
#  25550.0: 0.3621569945902312, 
#  25500.0: 0.5759808254168853, 
#  25600.0: 0.7142410721708303}

In [12]:
option_greeks(
    spot=25406,
    strike=25550,
    dte=6/365,
    rate=0.086,
    option_type='PE',
    option_price=175.25
)

(np.float64(0.0869345443754492),
 np.float64(0.6460475040319663),
 np.float64(0.0022814270363698415))